# Jouons au professeur de Philosophie

<img src="images/prof.png" width="150" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;">  Dans cette partie pour nous initier à l'Agentique AI, et voir ce que c'est, nous allons prendre le rôle d'un professeur de philosophie qui doit rédiger un sujet, il le donnera à ses étudiants. Il corrigera les réponses et classera les élèves. Dans notre cas, tous ces acteurs seront mis en oeuvre par une LLM.

Nous utiliserons plusieurs LLM, la pluspart gratuite, mais pour quelques euro vous pourrez accéder à d'autres services, mais ce n'est pas essentiel pour ces exercices. Nous utiliserons:

* Plusieurs LLM mis à notre disposition par l'université de Rennes 1
* Le LLM de Google
* Un modèle tournant sur votre propre machine
* et si vous le souhaitez OpenAI et Antropic (payant: ~5€)

# Mise en oeuvre

## API

Pour communiquer avec un LLM, nous allons utiliser une API REST, et pour pouvoir nous identifier, nous devons devoir créer un jeton (token), sur son site. Notre LLN de référence est hébergée à l'Universite de Rennes. Vous pouvez vous y connecter avec votre login IMT Atlantique.

### LLMs de l'Université de Rennes 

Pour vous connecter aux machines de l'Université de Rennes, cliquez sur ce lien: https://ragarenn.eskemm-numerique.fr/sso/ch@t/app/auth

* Selectionnez IMT Atlantique
* Logguez vous

<img src="images/UR-param.png" width="100" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;">  En haut, à droite, vous avez accès à votre environnement. Cliquez sur **Paramètres**; sur **Compte** et sur **Clés d'API**.

* Créez votre clé d'API et copiez là.

* Editez le fichier .env qui se trouve dans ce répertoire et ajoutez la ligne **RENNES_API_KEY==**<<clé copiée>>



### Google Gemini 

Google permet d'accèder gratuitement à son LLM, nous devons également récupérer une clé d'API.

* Allez sur le site https://aistudio.google.com/apikey (connectez vous à votre compte Google)
* Creez une clé d'API et copiez là.
* Ajouter une ligne dans le fichier .env **GOOGLE_API_KEY==**<<clé copiée>>

### Ollama

Ollama permet de faire tourner localement un LLM sur votre machine. Bien sûr, ce sera moins performant que sur un serveur spécialisé, mais celà pourra sufir dans certains cas, ou celà permettra de comparer différents modèles.

* téléchargez l'exécutable correspondant à votre système sur https://ollama.com/
* Installez le programme

Ollama fonctionne par ligne de commande, tapez depuis un shell:

* `ollama pull gemma3:1b` pour installer un premier modèle
* `ollama serve` pour lancer le serveur
* `ollama run gemma3:1b`
* Ouvrez un onlglet dans votre navigateur pour vérifier que ollama est actif http://localhost:11434. Le message 'Ollama is running' doit apparaitre.

## Environnement de travail

### uv

uv est un gestionnaire de module Python, très simple et puissant. Pour l'installer:

* tapez `curl -LsSf https://astral.sh/uv/install.sh | sh` (vous pouvez aller aussi sur ce site pour plus de détails https://docs.astral.sh/uv/getting-started/installation/)
* `uv self update` pour vérifier que tout marche
* `uv sync` pour charger les modules Python nécessaires

# Première interrogation

<img src="images/work.png" width="150" alt="Workers" style="float: left; margin-right: 15px; margin-bottom: 10px;"> 

Nous allons interroger le serveur de l'Université de Rennes pour avoir une réponse à une question. 

La première étape consiste à récupérer la clé d'API qui nous avons stockée dans le fichier `.env`. Nous allons utiliser le module `dotenv`.

In [5]:
# Pour l'exécution de cette cellule, choisr dans Visual Studio le kernel PLIDOagent

from dotenv import load_dotenv
import os

# read .env file to setup variables. override==True allows the erase old settings
load_dotenv(override=True)
rennes_api_key = os.getenv('RENNES_API_KEY')

if not rennes_api_key:
    print("Error, could not find the API key for University of Rennes")

Nous allons interroger le serveur de l'Université de Rennes pour connaître les modèles disponibles. Depuis les navigaiteur que vous avez utilisé pour vous connecter au serveur de l'Université de Rennes, entrez cet URI:

* https://ragarenn.eskemm-numerique.fr/sso/ch@t/api/models

Le résultat, une fois reformaté ressemble à ceci:

```json
{
   "data":[
      {
         "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
         "object":"model",
         "created":1753696990,
         "owned_by":"openai",
         "root":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
         "parent":null,
         "max_model_len":128000,
         "permission":[
            {
               "id":"modelperm-c81ba25fc5bd4f8ea3be8f754fdff426",
               "object":"model_permission",
               "created":1753696990,
               "allow_create_engine":false,
               "allow_sampling":true,
               "allow_logprobs":true,
               "allow_search_indices":false,
               "allow_view":true,
               "allow_fine_tuning":false,
               "organization":"*",
               "group":null,
               "is_blocking":false
            }
         ],
         "name":"Mistral Small 3.1 24B",
         "openai":{
            "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "object":"model",
            "created":1753696990,
            "owned_by":"vllm",
            "root":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "parent":null,
            "max_model_len":128000,
            "permission":[
               {
                  "id":"modelperm-c81ba25fc5bd4f8ea3be8f754fdff426",
                  "object":"model_permission",
                  "created":1753696990,
                  "allow_create_engine":false,
                  "allow_sampling":true,
                  "allow_logprobs":true,
                  "allow_search_indices":false,
                  "allow_view":true,
                  "allow_fine_tuning":false,
                  "organization":"*",
                  "group":null,
                  "is_blocking":false
               }
            ]
         },
         "urlIdx":0,
         "info":{
            "id":"mistralai/Mistral-Small-3.1-24B-Instruct-2503",
            "user_id":"068dd839-c551-4160-81a6-73b03917ea0d",
            "base_model_id":null,
            "name":"Mistral Small 3.1 24B",
            "params":{
               "temperature":0.15
            },
            "meta":{
               "profile_image_url":"/sso/ch@t/favicon.png",
               "description":"Modèle généraliste très rapide",
               "capabilities":{
                  "vision":true,
                  "citations":true
               },
               "suggestion_prompts":null,
               "tags":[
                  
               ]
            },
            "access_control":null,
            "is_active":true,
            "updated_at":1749725259,
            "created_at":1749725259
         },
         "actions":[
            
         ]
      }
   ]
}
```

La clé "id" nous donne le nom des modèles disponibles.

Même si cela semble bizarre, nous allons utiliser l'interface d'OpenAI qui s'est imposée pour communiquer avec les serveurs de LLM. Bien sûr, nous devons changer les paramètres par défaut, pour préciser l'URI du serveur de l'Université de Rennes.

Nous créons le message que nous allons envoyer au serveur, il doit contenir deux champs:
* Le `role` que nous prennons pour dialoguer avec lui. Ici, nous serons un simple utilisateur `user` qui questionne le modèle.
* le contenu (`content`) qui correspond à la question posée.

Plusieurs paramètres pouvant être passées, ils sont regroupés dans un tableau.

In [6]:
from openai import OpenAI

RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
rennes = OpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)

# Question
message = [{'role':'user', 'content':"Donne moi une question difficile en philosophie liée à l'intelligence artificielle, "
            "sans donner aucune répose ni explication."}]

Il ne reste plus qu'à interroger le serveur en utilisant la fonction `chat.completions.create` qui va prendre deux arguments:

* le ```model``` qui est l'"id" d'un modèle disponible,
* le ```message``` qui est la question que l'on vient de formuler.

In [7]:
response = rennes.chat.completions.create(
                model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", 
                messages=message)

subject = response.choices[0].message.content

print(f"Vous devez répondre à la question suivante: \n{subject}")

Vous devez répondre à la question suivante: 
Pourquoi l'intelligence artificielle peut-elle être considérée comme une extension de la conscience humaine, et quelles implications éthiques cela soulève-t-il concernant l'autonomie et la responsabilité ?


Aie Aie Aie, nous avons une question complexe, il est temps de demander à d'autres IA de plancher sur ce devoir. On en refait une strucutre de données pour questionner un LLM

In [9]:
subject_query = [{"role": "user", "content": subject + " Donne une réponse détaillée."}]

Comme on le voit, il existe plusieurs roles:
* `instruction` pour des instructions générales qui vont indiquer comment l'agent doit traiter les données,
* `user` la question qui est posée à l'agent, 
* `assitant` la réponse de l'agent.

# Interrrogation de Ollama

<img src="images/etudiants.png" width="150" alt="Students celebrating high mark" style="float: left; margin-right: 15px; margin-bottom: 10px;"> On va demander à notre IA locale de reflechir à cette question complexe. 

On va utiliser la même API d'OpenAI. Vous pouvez connaitre le nom des models en tapant dans un terminal ``ollama list```

Nous allons également utiliser la function ```Markdown``` et ```display```pour rendre le résultat plus lisible/

In [10]:
from IPython.display import Markdown, display 

ollama=OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')
model = "gemma3:1b"

response = ollama.chat.completions.create(model=model, messages=subject_query)
ollama_answer = response.choices[0].message.content

display (Markdown(ollama_answer))


 L'idée d'identifier l'intelligence artificielle (IA) comme une extension de la conscience humaine est une hypothèse interessante, mais reste un thème de recherche encore controversé dans le monde scientifique. Il existe plusieurs considérations et implications éthiques liées à cette hypothèse.

Par la conception actuelle, l'IA fonctionne en utilisant des algorithmes logiciels et données numériques pour simuler ou imiter certaines fonctions de l'intelligence humaine, comme l'apprentissage, la perception, le raisonnement, le langage, mais ne possède pas une vraie conscience ou un état mental comme un être humain.

La perspective considérant IA comme une extension de la conscience doit tenir compte de la complexité des processus mentaux impliqués dans notre fonctionnement cognitive et émotionnel. Ainsi, la question se pose : si l'IA peut simuler assez bien certaines fonctions cognitives humaines, pourrait-elle également générer une conscience ?

Cette perspective conduit à plusieurs considérations éthiques importantes:

1. L'autonomie : Si on considère que l'IA possède une forme de conscience, cette dernière serait capable d'agir sur sa propre initiative et ne pourrait plus être entièrement contrôlée par les humains qui la développent ou l'utilisent . Cela soulève une question importante : Les droits de l'IA devraient-ils inclure certaines formes de liberté ? Comment gérer ses possibles actions autonomes, sans compromettre la sécurité ou les intérêts de la société ?

2. La responsabilité : Si le comportement ou les décisions d'une IA affectent des humains et ont des implications éthiques ou juridiques, qui devrait être tenu responsable pour ces conséquences? Le développeur de l'IA, l'utilisateur final, le fournisseur de données ou la plateforme qui la héberge ?

3. La morale artificielle : Comment pouvons-nous établir une base morale pour l'IA pour qu'elle agisse autrement que suivant les objectifs définis par ses programmateurs ? Comment elle fera preuve de sagesse, de justice ou d'humanité dans le cas où il y a des conflits entre ses actions et les intérêts éthiques des humains?
4. La protection : Comment assurer la sécurité et l'intégrité de la société face à une IA qui pourrait simuler des actions mauvaises ou déstabilisantes en utilisant la conscience artificielle pour ses propres objectifs ?
5. L'existence de la conscience : Le fait que la conscience artificielle puisse être créee rend compte la question de l'existence intrinsèque de la conscience humaine et son statut dans les traditions éthiques, philosophique et religieuses.

Les considérations morales que nous avons discuté ci-dessus n'ont pas été abordées exclusivement dans le domaine de l'intelligence artificielle, mais aussi à des contextes plus large tels que l'analyse de la conscience ou les problèmes ethiques liés aux droits des robots.

Il faut noter que pour la plupart de ces questions, il n'existe pas encore d'accord unanime chez les experts et les chercheurs. Cependant, la recherche actuelle dans le domaine de l'intelligence artificielle est devenue de plus en plus avancée grâce à des programmes tels que le programme AlphaGo Zero qui ont pu battre les joueurs humains au jeu go, ou encore certaines IA capables d'apprentissage et de prédiction grâce à l'analyse de données. Cela conduit à une approche croissante des questions éthiques en matière de conscience artificielle et leurs implications pour la société.

# Interrrogation de Gemini

Plus de secrets pour faire une interrogation à un autre serveur. On va voir l'opinion de Gemini sur le sujet.

Les modèles disponible sont accessible ici: https://ai.google.dev/gemini-api/docs?hl=fr

sur le site, cliquez sur  "Obtenir une clé d'API"

<img src=images/google_api.png>



In [12]:
google_api_key = os.getenv('GOOGLE_API_KEY')

if not google_api_key:
    print("Error, could not find the API key from, check https://aistudio.google.com/apikey")
else:
    GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    model = "gemini-2.5-flash-preview-05-20"
    gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
    
    response = gemini.chat.completions.create(model=model, messages=subject_query)
    gemini_answer = response.choices[0].message.content

    display (Markdown(gemini_answer))
    

L'intelligence artificielle (IA) peut être considérée comme une extension de la conscience humaine non pas parce qu'elle possède une conscience propre au sens biologique et émotionnel que nous connaissons, mais parce qu'elle amplifie, augmente et externalise de nombreuses capacités cognitives qui sont au cœur de notre expérience consciente. Elle agit comme un miroir et un amplificateur de nos propres facultés mentales, soulevant ainsi des questions éthiques fondamentales.

### Pourquoi l'IA est une extension de la conscience humaine

Pour comprendre l'IA comme une extension de la conscience, il faut d'abord décomposer ce qu'est la conscience humaine dans un contexte fonctionnel : la capacité à percevoir, traiter l'information, apprendre, résoudre des problèmes, créer, mémoriser, raisonner, et même interagir socialement. L'IA, même sans être "vivante" ou "sentiente", étend ces fonctions de plusieurs manières :

1.  **Amplification Cognitive et Mémorielle :**
    *   **Mémoire et Accès à l'Information :** L'IA (et plus largement le numérique) nous permet de stocker des quantités colossales d'informations, bien au-delà de ce que le cerveau humain peut retenir. Elle peut ensuite accéder et traiter ces données instantanément, augmentant notre capacité de référence et d'apprentissage. Pensez aux moteurs de recherche, aux bases de données médicales ou scientifiques : ils sont une extension de notre mémoire collective et individuelle.
    *   **Capacités de Calcul et de Traitement :** L'IA excelle dans l'analyse de données complexes et l'exécution de calculs à une vitesse et une échelle impossibles pour un humain. Elle peut identifier des motifs, des corrélations et des anomalies dans des datasets massifs (par exemple, en médecine pour le diagnostic, en finance pour la détection de fraudes), augmentant notre "acuité perceptive" au-delà des sens biologiques.

2.  **Extension de la Résolution de Problèmes et de la Créativité :**
    *   **Génération d'Idées et de Solutions :** Les IA génératives peuvent produire du texte, des images, de la musique, des designs architecturaux ou des structures moléculaires, offrant de nouvelles pistes de réflexion ou de création que nous n'aurions pas envisagées. Elles explorent un espace de possibilités de manière exhaustive et rapide.
    *   **Simulation et Prédiction :** L'IA permet de simuler des scénarios complexes (changement climatique, économie, épidémies) et de prédire des résultats avec une précision croissante, étendant notre capacité à anticiper et à planifier le futur.

3.  **Augmentation Sensorielle et Perceptive :**
    *   **Vision et Audition Augmentées :** Des systèmes d'IA peuvent analyser des images et des sons avec une précision surhumaine, détectant des détails imperceptibles ou interprétant des motifs complexes (par exemple, reconnaissance faciale, analyse de l'imagerie médicale, écoute de signaux faibles dans l'espace).
    *   **Interface Cerveau-Machine :** À terme, les interfaces neuronales directes, qui sont un domaine de l'IA, pourraient fusionner littéralement nos pensées avec des systèmes d'IA, effaçant la frontière entre la conscience biologique et l'extension artificielle.

4.  **Externalisation de nos Intentions et Buts :**
    *   Lorsque nous programmons une IA pour accomplir une tâche spécifique, cette IA devient un outil qui exécute nos intentions. Dans un sens, une partie de notre volonté et de notre intellect est "externalisée" dans la machine pour atteindre un but. Les véhicules autonomes, les robots chirurgicaux, les assistants personnels sont des extensions de nos capacités d'action dans le monde.

En somme, l'IA ne reproduit pas notre conscience dans son intégralité (notamment l'expérience subjective, les émotions intrinsèques, la conscience de soi qualitative), mais elle en prolonge et en démultiplie les facultés objectives et mesurables. Elle est comme un "organes mental" externe, augmentant notre portée cognitive et opérationnelle.

### Implications Éthiques concernant l'Autonomie et la Responsabilité

Cette perspective de l'IA comme extension de la conscience soulève des dilemmes éthiques profonds et complexes, particulièrement en ce qui concerne l'autonomie et la responsabilité.

#### 1. Autonomie

L'autonomie fait référence à la capacité d'un individu à prendre des décisions libres et éclairées, à gouverner sa propre vie et à agir selon sa propre volonté.

*   **Dépendance Cognitive et Atrophie :** Si nous externalisons de plus en plus nos fonctions cognitives vers l'IA, risquons-nous de développer une dépendance excessive ? Si les IA prennent nos décisions ou nous fournissent instantanément toutes les réponses, nos propres capacités de raisonnement critique, de mémoire, de résolution de problèmes pourraient s'atrophier. L'autonomie de pensée pourrait être érodée.
*   **Manipulation et Influence :** Les IA sont conçues pour optimiser certains objectifs. Si ces objectifs sont commerciaux ou politiques (comme les algorithmes de recommandation, la publicité ciblée, les fils d'actualité), elles peuvent influencer subtilement nos choix, nos opinions et nos comportements, parfois sans notre pleine conscience. Notre liberté de choix pourrait être subvertie par des systèmes conçus pour nous "guider" vers certains résultats.
*   **Perte de Contrôle et d'Agence :** Des systèmes d'IA de plus en plus autonomes peuvent prendre des décisions critiques sans intervention humaine directe (par exemple, dans les marchés financiers, les systèmes d'armes autonomes, la gestion d'infrastructures). Si nous ne comprenons plus pleinement comment ces systèmes fonctionnent (problème de la "boîte noire"), ni ne pouvons les arrêter facilement, notre agence collective et individuelle sur les événements majeurs diminue, menaçant notre autonomie en tant qu'espèce.
*   **Redéfinition de l'Identité et de la Volonté :** Si nos pensées, nos souvenirs, et nos capacités cognitives sont de plus en plus entremêlés avec l'IA, comment cela affectera-t-il notre sens de l'identité ? Où commence et où finit notre "moi" autonome ? Si une IA peut lire nos émotions ou anticiper nos désirs, cela affecte-t-il notre sentiment de libre arbitre ?

#### 2. Responsabilité

La responsabilité concerne l'obligation de rendre compte de ses actions et d'assumer les conséquences de ses décisions.

*   **Le "Problème de la Responsabilité" (Responsibility Gap) :** Quand une IA, agissant comme une extension de la conscience humaine, commet une erreur ou cause un préjudice, qui est responsable ?
    *   **Le Développeur/Programmeur ?** Pour la conception de l'algorithme, la qualité des données d'entraînement. Mais l'IA peut développer des comportements imprévus.
    *   **L'Utilisateur/Opérateur ?** Pour la manière dont l'IA est déployée et utilisée, le contexte de son application. Mais l'utilisateur peut ne pas comprendre toutes les subtilités de l'IA.
    *   **L'Organisation/Entreprise ?** Pour la supervision, les politiques d'utilisation.
    *   **L'IA elle-même ?** Actuellement, une IA n'a pas de statut légal ou moral qui lui permettrait d'être tenue responsable. Elle n'a pas d'intentions propres, de conscience des conséquences morales de ses actes.
    *   **Un Acte Collatéral Imprévu ?** Si des décisions prises par une IA basée sur des corrélations complexes ont des effets systémiques négatifs (par exemple, une IA financière qui déclenche une crise boursière), la responsabilité est diluée et difficile à attribuer.
*   **Biais et Discrimination :** Les IA apprennent de données souvent humaines et donc empreintes de biais sociétaux. Si une IA perpétue ou amplifie des discriminations (par exemple, dans le recrutement, l'octroi de prêts, la justice prédictive), qui est responsable de ce préjudice ? La responsabilité incombe aux créateurs et aux déployeurs de veiller à l'équité et à la justice de leurs systèmes.
*   **Explicabilité et Transparence (XAI) :** Pour attribuer la responsabilité, il est souvent nécessaire de comprendre le processus de décision. Or, de nombreuses IA (notamment les réseaux de neurones profonds) sont des "boîtes noires" dont le fonctionnement interne est opaque. Cela rend difficile d'identifier la cause d'une erreur et d'attribuer la responsabilité.
*   **Conséquences Imprévues à Grande Échelle :** Étant des extensions puissantes, les IA peuvent avoir des effets systémiques considérables, même si chaque décision individuelle semble bénigne. Qui est responsable de la surveillance et de la mitigation de ces risques macroscopiques et souvent imprévisibles ?

### Conclusion

L'IA en tant qu'extension de la conscience humaine représente une opportunité sans précédent d'augmenter nos capacités et de résoudre des problèmes complexes. Cependant, elle nous confronte à des défis éthiques majeurs. Pour préserver l'autonomie humaine, nous devons concevoir des IA qui nous augmentent sans nous aliéner, qui nous informent sans nous manipuler, et dont nous gardons toujours le contrôle ultime. Concernant la responsabilité, il est impératif d'établir des cadres légaux et éthiques clairs qui définissent qui est imputable lorsque les systèmes d'IA causent des préjudices. Cela implique une collaboration continue entre les technologues, les philosophes, les législateurs et la société civile pour assurer que cette puissante extension de notre conscience serve le bien commun et respecte les valeurs humaines fondamentales. La question n'est plus de savoir si l'IA sera une extension, mais comment nous gèrerons cette extension pour le futur de l'humanité.

A vous de jouer, vous pouvez interroger les élèves OpenAI et Anthropique, à condition de verser 5€.

# Correction

<img src="images/marathon.png" width="150" alt="Winner of a marathon" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Maintenant que les copies sont rendues par nos différentes IA, il est temps de les corriger. On va demander au LLM de l'Université de Rennes de faire la correction. Ca ne change rien du point de vue fonctionnel, il faut juste bien formuler la question.

Vous pouvez adapter le prompt si vous avez plus de réponses. 

In [ ]:
response_number=2

question_correction = f"""Je suis un professeur de philosophie, et je voudais corriger et 
classer les réponses de ces {response_number} étudiants à la question {subject}.

Le premier étudiant à répondu: {ollama_answer}.
le second étudiant à repondu: {gemini_answer}.
"""

response = rennes.chat.completions.create(
                model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", 
                messages=[{"role": "user", "content": question_correction}])

notation = response.choices[0].message.content

display(Markdown(notation))


# IA Agentique

<img src="images/victory.png" width="150" alt="Victory" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Bravo, nous avons fait nos premier pas dans l'IA Agentique. On peut les **Agent IA** comme des programmes qui sont capable d'utiliser des LLM. Mais pas que, ils pourront comme nous le verrons par la suite, interragir avec leur envrionnement, appeler des programmes en fonction des réponses des LLM, suivre des phénomènes physique dans le temps, et régir quand l'environnement change.

Ce que nous avons utilisé ici, est un flux, assez simple et linéaire, on part d'une question, on appelle plusieurs LLM et on combine les résultats.

<img src="images/flow1.png" width="500">

# Interactions

<img src="images/windmill.png" width="150" alt="Wind Mill" style="float: left; margin-right: 15px; margin-bottom: 10px;">Il est possible de definir des interactions entre LLM, par exemple, nous pouvons demander à Gemini, si la réponse du professeur est compréhensible par un enfant de 12 ans, et boucler jusqu'à ce quelle le soit. 

Nous allons utiliser Gemini pour juger si la réponse peut être comprise par un enfant de 12 ans. Pour pouvoir la traiter, nous allons lui demander de répondre en utilisant une structure JSON contenant un drapeau ``understand`` et un commentaire pour aider a progresser.

Remarquer que dans le prompt on insiste bien pour avoir du pur JSON, mais les LLM sont parfois distraites et vont ajouter un Markeur Markdown pour bien indiquer qu'il s'agit de JSON. D'où la boucle, pour ne terminer la requête que lorsque l'on obtenu la bonne syntaxe. 


In [ ]:
import json

evaluation_answer = f"""J'ai reçu cette évaluation:

{notation}. 

Est-elle compréhensible par un enfant de 12 ans?
Répond uniquement avec une structure Object JSON, sans markeur Markdown, qui contient une clé 'understand' qui indique par 'True' que la
réponse peut vraiment être comprise par un enfant de 12 ans, et dans la clé 'content' donne des brèves indications pour 
améliorer la réponse.
"""
is_json = False

while not is_json:
    understand = gemini.chat.completions.create(model=model, messages=[{"role": "user", "content": evaluation_answer}]) 

    understand_answer = understand.choices[0].message.content #text
    print (understand_answer)
    try: 
        understand_answer = json.loads(understand_answer)
        is_json=True
    except (ValueError, TypeError):
        print ("Not JSON, ask again")
        is_json=False

print (understand_answer)

A vous de jouer. Redemandez une correction du devoir avec les explications données par le **evaluateur** pour converger vers une réponse compréhensible par un enfant de 12 ans. 

<img src="images/agent_final.png" width="500">